In [1]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import requests
import pandas as pd

In [2]:
url_base = 'https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/mazowieckie/warszawa/warszawa/warszawa?limit=72&ownerTypeSingleSelect=ALL&distanceRadius=0&by=DEFAULT&direction=DESC&viewType=listing&page='
num_pages = 50

headers = {'User-Agent': 'Mozilla/5.0'}

all_data = []

for page_num in range(1, num_pages + 1):
    url = url_base + str(page_num)

    req = Request(url, headers=headers)

    try:
        response = urlopen(req, timeout=10)
        page = response.read()

        soup = BeautifulSoup(page, 'html.parser')

        for t, p, l, a in zip(
            soup.find_all('div', {'class': 'css-gg4vpm e1n06ry51'}),
            soup.find_all('div', {'class': 'e1jyrtvq0 css-1tjkj49 ei6hyam0'}),
            soup.find_all('p', {'class': 'css-19dkezj e1n06ry53'}),
            soup.find_all('span', {'class': 'css-1ygypsy enzg89n5'})
        ):
            data = {
                'Listing': t.text.strip(),
                'Price1': p.text.strip(),
                'Location': l.text.strip(),
                'Ads': a.text.strip(),
            }
            all_data.append(data)

        print(f'Scraping page {page_num}')

    except Exception as e:
        print(f'Error: {e}')

Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
Scraping page 11
Scraping page 12
Scraping page 13
Scraping page 14
Scraping page 15
Scraping page 16
Scraping page 17
Scraping page 18
Scraping page 19
Scraping page 20
Scraping page 21
Scraping page 22
Scraping page 23
Scraping page 24
Scraping page 25
Scraping page 26
Scraping page 27
Scraping page 28
Scraping page 29
Scraping page 30
Scraping page 31
Scraping page 32
Scraping page 33
Scraping page 34
Scraping page 35
Scraping page 36
Scraping page 37
Scraping page 38
Scraping page 39
Scraping page 40
Scraping page 41
Scraping page 42
Scraping page 43
Scraping page 44
Scraping page 45
Scraping page 46
Scraping page 47
Scraping page 48
Scraping page 49
Scraping page 50


In [3]:
df = pd.DataFrame(all_data)
df

,Listing,Price1,Location,Ads
0,"Mieszkanie 55m2 po remoncie, balkon Mokotów",820 000 zł14 909 zł/m²3 pokoje55 m²,"Stary Mokotów, Mokotów, Warszawa, mazowieckie",Biuro nieruchomości
1,Sprzedam 4 pokojowe 96 m na Mokotowie,1 400 000 zł14 583 zł/m²4 pokoje96 m²,"Wierzbno, Mokotów, Warszawa, mazowieckie",Biuro nieruchomości
2,Do odświeżenia,780 000 zł10 894 zł/m²3 pokoje71.6 m²,"Grodzisk, Białołęka, Warszawa, mazowieckie",Biuro nieruchomości
3,REZERWACJA! Idealne dla Pary lub na Inwestycje,560 000 zł12 444 zł/m²2 pokoje45 m²,"Stary Żoliborz, Żoliborz, Warszawa, mazowieckie",Biuro nieruchomości
4,Mieszkanie Mokotów,720 000 zł17 143 zł/m²2 pokoje42 m²,"Służew, Mokotów, Warszawa, mazowieckie",Biuro nieruchomości
...,...,...,...,...
2839,"Mieszkanie po remoncie 37,50 m2 Mokotów",570 000 zł15 200 zł/m²2 pokoje37.5 m²,"al. Aleja Lotników, Służew, Mokotów, Warszawa,...",Biuro nieruchomości
2840,Targówek przy metrze odnowione z widokiem na park,777 000 zł14 127 zł/m²3 pokoje55 m²,"ul. Złotopolska, Targówek Mieszkaniowy, Targów...",Biuro nieruchomości
2841,"Stary Mokotów, 4 pokoje, metro, garaż",955 000 zł14 692 zł/m²4 pokoje65 m²,"ul. Giordana Bruna, Stary Mokotów, Mokotów, Wa...",Biuro nieruchomości
2842,3 Pokoje kameralne osiedle w zieleni Stary Imi...,995 000 zł14 654 zł/m²3 pokoje67.9 m²,"ul. Puławska, Stary Imielin, Ursynów, Warszawa...",Biuro nieruchomości


In [4]:
divide = df['Price1'].str.split('zł', 1, expand=True)
divide.columns = ['Price', 'SQMUP1']

divide2 = divide['SQMUP1'].str.split('zł/m²', expand=True)
divide2.columns = ['SQMUP', 'Room Info1']

divide3 = divide2['Room Info1'].str.replace('pokój|pokoje', 'room', regex=True).str.split('room', expand=True)
divide3.columns = ['Room Info', 'Area']

divide3['Area'] = divide3['Area'].str.replace('m²', ' ')

In [5]:
divide_df = pd.concat([divide, divide2, divide3], axis=1)
divide_df.drop(columns=['SQMUP1', 'Room Info1'], inplace=True)
divide_df.head()

,Price,SQMUP,Room Info,Area
0,820 000,14 909,3,55
1,1 400 000,14 583,4,96
2,780 000,10 894,3,71.6
3,560 000,12 444,2,45
4,720 000,17 143,2,42


In [6]:
final_df = pd.concat([df, divide_df], axis=1)
final_df.drop(columns=['Price1'], inplace=True)
final_df = final_df[['Listing', 'Price', 'SQMUP', 'Room Info', 'Area', 'Location', 'Ads']]

final_df.head()

,Listing,Price,SQMUP,Room Info,Area,Location,Ads
0,"Mieszkanie 55m2 po remoncie, balkon Mokotów",820 000,14 909,3,55,"Stary Mokotów, Mokotów, Warszawa, mazowieckie",Biuro nieruchomości
1,Sprzedam 4 pokojowe 96 m na Mokotowie,1 400 000,14 583,4,96,"Wierzbno, Mokotów, Warszawa, mazowieckie",Biuro nieruchomości
2,Do odświeżenia,780 000,10 894,3,71.6,"Grodzisk, Białołęka, Warszawa, mazowieckie",Biuro nieruchomości
3,REZERWACJA! Idealne dla Pary lub na Inwestycje,560 000,12 444,2,45,"Stary Żoliborz, Żoliborz, Warszawa, mazowieckie",Biuro nieruchomości
4,Mieszkanie Mokotów,720 000,17 143,2,42,"Służew, Mokotów, Warszawa, mazowieckie",Biuro nieruchomości
